In [2]:
from sklearn import model_selection
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline   

In [37]:
Inhibitor = pd.read_csv("Inhibitor.csv")
NoInhibitor = pd.read_csv("NoInhibitor.csv")

In [38]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(NoInhibitor, Inhibitor, test_size=0.2, random_state=1)

In [39]:
import numpy as np
import pandas as pd
import tensorflow as tf
import datetime
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

In [72]:
def model_builder(hp):
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Flatten(input_shape=(160,)))

  hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  for i in hp.Int('mlplayers', minlayers = 4, maxlayers = 8):
    model.add(tf.keras.layers.Dense(
      units=hp.Int(f"UnitsintheLayer{i}", min_value=32, max_value=512, step=32), activation=hp_activation))
    
    
  model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=['accuracy'])
  
  return model

In [65]:
import keras_tuner as kt

tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=50,
                     factor=3,
                     directory='dir',
                     project_name='x')

INFO:tensorflow:Reloading Oracle from existing project dir/x/oracle.json
INFO:tensorflow:Reloading Tuner from dir/x/tuner0.json


In [74]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
tuner.search(x_train, y_train, epochs=80, max_trials = 600, validation_split=0.2, callbacks=[stop_early])

INFO:tensorflow:Oracle triggered exit


In [67]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [68]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(x_train, y_train, epochs=50, validation_split=0.2,
                    callbacks=[stop_early])

Epoch 1/50
9/9 [==============================] - 1s 27ms/step - loss: 1.0794 - accuracy: 0.5328 - val_loss: 0.6164 - val_accuracy: 0.6154
Epoch 2/50
9/9 [==============================] - 0s 12ms/step - loss: 0.5013 - accuracy: 0.8224 - val_loss: 0.4394 - val_accuracy: 0.9077
Epoch 3/50
9/9 [==============================] - 0s 14ms/step - loss: 0.3570 - accuracy: 0.9382 - val_loss: 0.3889 - val_accuracy: 0.8923
Epoch 4/50
9/9 [==============================] - 0s 14ms/step - loss: 0.3017 - accuracy: 0.9498 - val_loss: 0.3681 - val_accuracy: 0.8308
Epoch 5/50
9/9 [==============================] - 0s 10ms/step - loss: 0.2685 - accuracy: 0.9537 - val_loss: 0.3514 - val_accuracy: 0.9077
Epoch 6/50
9/9 [==============================] - 0s 14ms/step - loss: 0.2281 - accuracy: 0.9498 - val_loss: 0.3136 - val_accuracy: 0.8769
Epoch 7/50
9/9 [==============================] - 0s 12ms/step - loss: 0.2041 - accuracy: 0.9575 - val_loss: 0.2906 - val_accuracy: 0.9077
Epoch 8/50
9/9 [===========

In [71]:
history.history
import pandas as pd

pd.DataFrame(history.history)

,loss,accuracy,val_loss,val_accuracy
0,1.079365,0.532819,0.616377,0.615385
1,0.501257,0.822394,0.439407,0.907692
2,0.356992,0.938224,0.388948,0.892308
3,0.301725,0.949807,0.368067,0.830769
4,0.268548,0.953668,0.351374,0.907692
5,0.228074,0.949807,0.313623,0.876923
6,0.204068,0.957529,0.290587,0.907692
7,0.187722,0.953668,0.278946,0.938462
8,0.153362,0.969112,0.252930,0.907692
9,0.133882,0.965251,0.251639,0.923077
